In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras

In [3]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()
X_train[0][:10]

17473536/17464789 [==============================] - 0s 0us/step


[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [6]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
  id_to_word[id_] = token

" ".join([id_to_word[id_] for id_ in X_train[0][:10]  ]) 

'<sos> this film was just brilliant casting location scenery story'

In [10]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteL2NT16/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteL2NT16/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [14]:
def preprocess(X_batch, y_batch):
  X_batch = tf.strings.substr(X_batch, 0, 300)
  X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
  X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
  X_batch = tf.strings.split(X_batch)
  return X_batch.to_tensor(default_value=b"<pad>"), y_batch


In [17]:
from collections import Counter
vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
  for review in X_batch:
    vocabulary.update(list(review.numpy()))

In [20]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [21]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

In [23]:
words = tf.constant(truncated_vocabulary)
word_ids=tf.range(len(truncated_vocabulary), dtype = tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [24]:
table.lookup(tf.constant([b"This movie was faaanantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10693]])>

In [25]:
def encode_words(X_batch, y_batch):
  return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)  

In [28]:
embed_size = 128
model = keras.models.Sequential([
                                keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                                                       input_shape=[None]),
                                keras.layers.GRU(128, return_sequences=True),
                                keras.layers.GRU(128),
                                keras.layers.Dense(1, activation="sigmoid")

])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 123s 148ms/step - loss: 0.6853 - accuracy: 0.5384
Epoch 2/5
782/782 [==============================] - 115s 147ms/step - loss: 0.4484 - accuracy: 0.7878
Epoch 3/5
782/782 [==============================] - 115s 146ms/step - loss: 0.3049 - accuracy: 0.8732
Epoch 4/5
782/782 [==============================] - 115s 147ms/step - loss: 0.2158 - accuracy: 0.9189
Epoch 5/5
782/782 [==============================] - 115s 147ms/step - loss: 0.1480 - accuracy: 0.9478


In [33]:
K = keras.backend
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs : K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.Model(inputs=[inputs], outputs=[outputs])

In [36]:
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [37]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size=32
train_set = datasets["train"].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 7s 8ms/step - loss: 0.5520 - accuracy: 0.7188
Epoch 2/5
782/782 [==============================] - 6s 7ms/step - loss: 0.5141 - accuracy: 0.7480
Epoch 3/5
782/782 [==============================] - 5s 6ms/step - loss: 0.5087 - accuracy: 0.7518
Epoch 4/5
782/782 [==============================] - 5s 6ms/step - loss: 0.5047 - accuracy: 0.7538
Epoch 5/5
782/782 [==============================] - 5s 6ms/step - loss: 0.5016 - accuracy: 0.7558


In [43]:
test_set = datasets["test"].batch(batch_size).prefetch(1)
scores = model.evaluate(test_set)

782/782 [==============================] - 5s 6ms/step - loss: 0.5124 - accuracy: 0.7466
